# Particle Swarm Optimisation for Knights Covering Problem

In [1]:
#necessary python library imports
import numpy as np
from collections.abc import Callable

## Algorithm Implementation

### Particle Swarm Optimisation class

In [1]:
class PSO:
    def __init__(self, board_size: int, fitness_function: Callable[[np.ndarray], np.ndarray], 
                 penalty_function: Callable[[np.ndarray, int], np.ndarray] = (lambda _: 0), num_particles: int = 100, 
                 c1: float = 1.0, c2: float = 1.0, max_velocity: float = 4.0, inertia: float = 1.0, rng_seed: int = None) -> None:
        
        #check for sensible hyperparameter selection
        assert max_velocity > 0, "Max velocity has to be greater than 0"
        assert num_particles > 1, "There has to be at least one particle"
        assert c1 >= 0, "c1 has to be greater than 0"
        assert c2 >= 0, "c2 has to be greater than 0"
        assert board_size > 3, "board size has to be at least 3"
        #condition on sensible inertia values as proposed in: "F. van den Bergh. An Analysis of Particle Swarm Optimizers (2002)"
        assert inertia > c1+c2 / 2 - 1 and inertia >= 0, "Inertia must be positive and greater than (c1 + c2) / 2 -1"

        #assign hyperparameters to object variables
        self.board_size = board_size
        self.fitness_function = fitness_function
        self.penalty_function = penalty_function
        self.num_particles = num_particles
        self.c1 = c1
        self.c2 = c2
        self.max_velocity = max_velocity
        self.inertia = inertia
        self.rng = np.random.default_rng(rng_seed)

        #initialise particles and velocities
        self.particle_positions = self.rng.integers(low=0, high=1, endpoint=True, size=(self.num_particles, self.board_size**2))
        self.particle_best_pos = self.particle_positions.copy()
        self.particle_velocities = self.rng.random(size=(self.num_particles, self.board_size**2)) * self.max_velocity
        
        #initialise global best
        self.particle_fitness = self.compute_penalty_fitness(self.particle_positions, 0)
        self.particle_best_fitness = self.particle_fitness.copy()

        best_index = np.argmax(self.particle_fitness)
        self.global_best_location = self.particle_positions[best_index]
        self.global_best_fitness = self.particle_fitness[best_index]
    
    def compute_penalty_fitness(self, particle_positions: np.ndarray, k: int):
        ''' computes the fitness subtracted by the penalty for invalid solutions (i.e. not all squares attacked or occupied).
            The penalty function receives a parameter 'k', which indicates the current iteration number.
        '''
        return self.fitness_function(particle_positions) + self.penalty_function(particle_positions, k)

    def optimise(num_iterations = 100):
        for i in range(num_iterations):
            pass

### Fitness functions

### Penalty functions

## Experimentation

### Utility functions

### Experimentation config

### Run experiments

## Results

### Load results file

### Visualisation